# <img align="left" src="./images/movie_camera.png"     style=" width:40px;  " > 实践实验室：协同过滤推荐系统

在本练习中，您将实现协同过滤来构建电影推荐系统。 

# <img align="left" src="./images/film_reel.png"     style=" width:40px;  " > 大纲
- [ 1 - 符号](#1)
- [ 2 - 推荐系统](#2)
- [ 3 - 电影评分数据集](#3)
- [ 4 - 协同过滤学习算法](#4)
  - [ 4.1 协同过滤成本函数](#4.1)
    - [ 练习 1](#ex01)
- [ 5 - 学习电影推荐](#5)
- [ 6 - 推荐](#6)
- [ 7 - 恭喜！](#7)




##  包 <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;   " >
我们将使用熟悉的 NumPy 和 Tensorflow 包。

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

<a name="1"></a>
## 1 - 符号


|General <br />  Notation  | Description| Python (if any) |
|:-------------|:------------------------------------------------------------||
| $r(i,j)$     | scalar; = 1  if user j rated game i  = 0  otherwise             ||
| $y(i,j)$     | scalar; = rating given by user j on game  i    (if r(i,j) = 1 is defined) ||
|$\mathbf{w}^{(j)}$ | vector; parameters for user j ||
|$b^{(j)}$     |  scalar; parameter for user j ||
| $\mathbf{x}^{(i)}$ |   vector; feature ratings for movie i        ||     
| $n_u$        | number of users |num_users|
| $n_m$        | number of movies | num_movies |
| $n$          | number of features | num_features                    |
| $\mathbf{X}$ |  matrix of vectors $\mathbf{x}^{(i)}$         | X |
| $\mathbf{W}$ |  matrix of vectors $\mathbf{w}^{(j)}$         | W |
| $\mathbf{b}$ |  vector of bias parameters $b^{(j)}$ | b |
| $\mathbf{R}$ | matrix of elements $r(i,j)$                    | R |



<a name="2"></a>
## 2 - 推荐系统 <img align="left" src="./images/film_rating.png" style=" width:40px;  " >
在本实验中，您将实现协同过滤学习算法并将其应用于电影评分数据集。
协同过滤推荐系统的目标是生成两个向量：对于每个用户，一个体现用户电影品味的"参数向量"。对于每部电影，一个相同大小的特征向量，体现电影的某些描述。两个向量的点积加上偏置项应该产生用户可能给该电影的评分估计。

下面的图表详细说明了这些向量是如何学习的。

<figure>
   <img src="./images/ColabFilterLearn.PNG"  style="width:740px;height:250px;" >
</figure>

现有评分以矩阵形式提供，如图所示。$Y$ 包含评分；0.5 到 5（含），步长为 0.5。如果电影未被评分，则为 0。$R$ 在已评分的电影位置为 1。电影在行中，用户在列中。每个用户有一个参数向量 $w^{user}$ 和偏置。每部电影有一个特征向量 $x^{movie}$。这些向量通过使用现有的用户/电影评分作为训练数据同时学习。上面显示了一个训练示例：$\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$。值得注意的是，特征向量 $x^{movie}$ 必须满足所有用户，而用户向量 $w^{user}$ 必须满足所有电影。这就是这种方法名称的来源——所有用户协作生成评分集。 

<figure>
   <img src="./images/ColabFilterUse.PNG"  style="width:640px;height:250px;" >
</figure>

一旦学习了特征向量和参数，就可以使用它们来预测用户可能如何评价未评分的电影。这在上面的图表中显示。该方程是预测用户一对电影零的评分示例。


在本练习中，您将实现计算协同过滤目标函数的 `cofiCostFunc` 函数。实现目标函数后，您将使用 TensorFlow 自定义训练循环来学习协同过滤的参数。第一步是详细说明将在实验中使用的数据集和数据结构。

<a name="3"></a>
## 3 - 电影评分数据集 <img align="left" src="./images/film_rating.png"     style=" width:40px;  " >
数据集来自 [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/) 数据集。   
[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

原始数据集有 600 个用户评分的 9000 部电影。数据集已缩小规模，专注于 2000 年以来的电影。该数据集包含 0.5 到 5 的评分，步长为 0.5。缩小后的数据集有 $n_u = 443$ 个用户和 $n_m= 4778$ 部电影。 

下面，您将把电影数据集加载到变量 $Y$ 和 $R$ 中。

矩阵 $Y$（一个 $n_m \times n_u$ 矩阵）存储评分 $y^{(i,j)}$。矩阵 $R$ 是一个二元值指示矩阵，如果用户 $j$ 对电影 $i$ 进行了评分，则 $R(i,j) = 1$，否则 $R(i,j)=0$。 

在本练习的这一部分，您还将使用矩阵 $\mathbf{X}$、$\mathbf{W}$ 和 $\mathbf{b}$： 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 

$\mathbf{X}$ 的第 $i$ 行对应于第 $i$ 部电影的特征向量 $x^{(i)}$，$\mathbf{W}$ 的第 $j$ 行对应于第 $j$ 个用户的参数向量 $\mathbf{w}^{(j)}$。$x^{(i)}$ 和 $\mathbf{w}^{(j)}$ 都是 $n$ 维向量。在本练习中，您将使用 $n=10$，因此，$\mathbf{x}^{(i)}$ 和 $\mathbf{w}^{(j)}$ 有 10 个元素。
相应地，$\mathbf{X}$ 是一个 $n_m \times 10$ 矩阵，$\mathbf{W}$ 是一个 $n_u \times 10$ 矩阵。

我们将首先加载电影评分数据集以了解数据结构。
我们将使用电影数据集加载 $Y$ 和 $R$。  
我们还将使用预计算值加载 $\mathbf{X}$、$\mathbf{W}$ 和 $\mathbf{b}$。这些值将在实验的后面学习，但我们将使用预计算值来开发成本模型。

In [2]:
#Load data
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


In [3]:
#  From the matrix, we can compute statistics like average rating.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

Average rating for movie 1 : 3.400 / 5


<a name="4"></a>
## 4 - 协同过滤学习算法 <img align="left" src="./images/film_filter.png"     style=" width:40px;  " >

现在，您将开始实现协同过滤学习算法。您将从实现目标函数开始。 

在电影推荐设置中，协同过滤算法考虑一组 $n$ 维参数向量
$\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$、$\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ 和 $b^{(0)},...,b^{(n_u-1)}$，其中模型预测用户 $j$ 对电影 $i$ 的评分为
$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(i)}$。给定一个由某些用户对某些电影的评分组成的数据集，您希望
学习参数向量 $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$、
$\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ 和 $b^{(0)},...,b^{(n_u-1)}$，以产生最佳拟合（最小化平方误差）。

您将完成 `cofiCostFunc` 中的代码以计算成本
function for collaborative filtering. 


<a name="4.1"></a>
### 4.1 协同过滤成本函数

协同过滤成本函数由下式给出
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\underbrace{
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}_{正则化}
\tag{1}$$
(1) 中的第一个求和是"对于所有 $i$、$j$，其中 $r(i,j)$ 等于 $1$"，可以写成：

$$
= \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\text{正则化}
$$

您现在应该编写 `cofiCostFunc`（协同过滤成本函数）以返回此成本。

<a name="ex01"></a>
### 练习 1

**For 循环实现**：   
首先使用 for 循环实现成本函数。
考虑分两步开发成本函数。首先，开发不带正则化的成本函数。下面提供了一个不包含正则化的测试用例来测试您的实现。一旦它工作正常，添加正则化并运行包含正则化的测试。请注意，只有当 $R(i,j) = 1$ 时，您才应该累加用户 $j$ 和电影 $i$ 的成本。

In [40]:
# GRADED FUNCTION: cofi_cost_func
# UNQ_C1

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += r * np.square((np.dot(w,x) + b_j - y ))
    J += (lambda_) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    J = J/2
    ### END CODE HERE ### 

    return J

In [41]:
# Public tests
from public_tests import *
test_cofi_cost_func(cofi_cost_func)

All tests passed!


<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示</b></font></summary>
    您可以使用两个 for 循环来构建代码，类似于 (1) 中的求和。   
    首先实现不带正则化的代码。   
    请注意，(1) 中的某些元素是向量。使用 np.dot()。您也可以使用 np.square()。
    请密切注意哪些元素由 i 索引，哪些元素由 j 索引。不要忘记除以二。
    
```python     
    ### START CODE HERE ###  
    for j in range(nu):
        
        
        for i in range(nm):
            
            
    ### END CODE HERE ### 
```    
<details>
    <summary><font size="2" color="darkblue"><b> 点击查看更多提示</b></font></summary>
        
    这里有一些更多细节。下面的代码在使用之前从矩阵中提取每个元素。 
    也可以直接引用矩阵。  
    此代码不包含正则化。
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = 
            y = 
            r =
            J += 
    J = J/2
    ### END CODE HERE ### 

```
    
<details>
    <summary><font size="2" color="darkblue"><b>最后手段（完整非正则化实现）</b></font></summary>
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    ### END CODE HERE ### 
```
    
<details>
    <summary><font size="2" color="darkblue"><b>正则化</b></font></summary>
     正则化只是将 W 数组和 X 数组的每个元素平方，然后对所有平方元素求和。
     您可以使用 np.square() 和 np.sum()。

<details>
    <summary><font size="2" color="darkblue"><b>正则化细节</b></font></summary>
    
```python 
    J += lambda_* (np.sum(np.square(W)) + np.sum(np.square(X)))
```
    
</details>
</details>
</details>
</details>

    


In [42]:
# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

Cost: 13.67


**Expected Output (lambda = 0)**:  
$13.67$.

In [43]:
# Evaluate cost function with regularization 
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 28.09


**Expected Output**:

28.09

**向量化实现**

创建向量化实现来计算 $J$ 很重要，因为它将在优化过程中被多次调用。所使用的线性代数不是本系列的重点，因此提供了实现。如果您是线性代数专家，可以自由创建自己的版本，而无需参考下面的代码。 

运行下面的代码并验证它产生与非向量化版本相同的结果。

In [44]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [45]:
# Evaluate cost function
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluate cost function with regularization 
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost: 13.67
Cost (with regularization): 28.09


**Expected Output**:  
Cost: 13.67  
Cost (with regularization): 28.09

<a name="5"></a>
## 5 - 学习电影推荐 <img align="left" src="./images/film_man_action.png" style=" width:40px;  " >
------------------------------

完成协同过滤成本函数的实现后，您可以开始训练算法为自己制作电影推荐。 

在下面的单元格中，您可以输入您自己的电影选择。然后算法将为您提供推荐！我们根据我们的偏好填写了一些值，但在您使用我们的选择使事情正常工作后，您应该更改此值以匹配您的口味。
数据集中所有电影的列表在文件 [电影列表](data/small_movie_list.csv) 中。

In [46]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[2700] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');


New user ratings:

Rated 5.0 for  Shrek (2001)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for  Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for  Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for  Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for  Incredibles, The (2004)
Rated 2.0 for  Persuasion (2007)
Rated 5.0 for  Toy Story 3 (2010)
Rated 3.0 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)
Rated 1.0 for  Nothing to Declare (Rien à déclarer) (2010)


现在，让我们将这些评论添加到 $Y$ 和 $R$ 中，并标准化评分。

In [47]:
# Reload ratings and add new ratings
Y, R = load_ratings_small()
Y    = np.c_[my_ratings, Y]
R    = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

Let's prepare to train the model. Initialize the parameters and select the Adam optimizer.

In [48]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

Let's now train the collaborative filtering model. This will learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$. 

同时学习 $w$、$b$ 和 $x$ 所涉及的操作不属于 TensorFlow 神经网络包提供的典型"层"。因此，课程 2 中使用的流程：Model、Compile()、Fit()、Predict()，不能直接应用。相反，我们可以使用自定义训练循环。

回想一下早期实验中的梯度下降步骤。
- 重复直到收敛：
    - 计算前向传播
    - 计算损失相对于参数的导数
    - 使用学习率和计算的导数更新参数 
    
TensorFlow 具有为您计算导数的出色能力。这如下所示。在 `tf.GradientTape()` 部分内，跟踪对 Tensorflow 变量的操作。当稍后调用 `tape.gradient()` 时，它将返回损失相对于被跟踪变量的梯度。然后可以使用优化器将梯度应用于参数。 
这是对 TensorFlow 和其他机器学习框架有用功能的非常简短的介绍。可以通过在所关注的框架内研究"自定义训练循环"来找到更多信息。
    


In [49]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136168.7
Training loss at iteration 40: 51863.3
Training loss at iteration 60: 24598.8
Training loss at iteration 80: 13630.4
Training loss at iteration 100: 8487.6
Training loss at iteration 120: 5807.7
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.2
Training loss at iteration 180: 2902.1


<a name="6"></a>
## 6 - 推荐
下面，我们计算所有电影和用户的评分，并显示推荐的电影。这些基于上面作为 `my_ratings[]` 输入的电影和评分。要预测用户 $j$ 对电影 $i$ 的评分，您计算 $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$。这可以使用矩阵乘法为所有评分计算。

In [50]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic

In practice, additional information can be utilized to enhance our predictions. Above, the predicted ratings for the first few hundred movies lie in a small range. We can augment the above by selecting from those top movies, movies that have high average ratings and movies with more than 20 ratings. This section uses a [Pandas](https://pandas.pydata.org/) data frame which has many handy sorting features.

In [51]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
1743,4.030965,4.252336,107,"Departed, The (2006)"
2112,3.985287,4.238255,149,"Dark Knight, The (2008)"
211,4.477792,4.122642,159,Memento (2000)
929,4.887053,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.796530,4.109091,55,Toy Story 3 (2010)
653,4.357304,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
1122,4.004469,4.006494,77,Shaun of the Dead (2004)
1841,3.980647,4.000000,61,Hot Fuzz (2007)
3083,4.084633,3.993421,76,"Dark Knight Rises, The (2012)"
2804,4.434171,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...


<a name="7"></a>
## 7 - 恭喜！ <img align="left" src="./images/film_award.png"     style=" width:40px;  " >
您已经实现了一个有用的推荐系统！